# Segmenting and Clustering Neighborhoods in Toronto

For this assignment, I will be exploring and clustering the neighborhoods in Toronto.

### 1) Data Cleaning and Preprocessing

Tasks:

1) Dataframe must consist of 3 columns: Postal Code, Borough, Neighborhood.

2) Only process cells that have a assigned borough - ignore borough that is not assigned.

3) More than 1 neighborhood can exist in 1 postal code area. Seperate this with columns to reduce amount of rows.

4) If cell has borough but no neighborhood, then borough name is also neighborhood.

In [383]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [384]:
#Read in data from url using read_html
data_table = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)
df = pd.DataFrame(data = data_table[0])
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [385]:
df.shape

(180, 3)

In [386]:
#Remove cells that do not have an assigned Borough.
df.fillna('Not assigned', inplace=True)

In [387]:
df1 = df[(df.Borough != 'Not assigned') & (df.Neighborhood != 'Not assigned')]
df1

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge
11,M3B,North York,Don Mills
12,M4B,East York,Parkview Hill / Woodbine Gardens
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [388]:
#Assign Borough name to neighborhood if neighborhood name is missing.

df1[(df1.Borough != 'Not assigned') & (df1.Neighborhood == 'Not assigned')]

,Postal code,Borough,Neighborhood


There are no missing Neighborhood names therefore no replation needs to be taken place.

In [389]:
df1.shape

(103, 3)

In [390]:
#Group all borough and postal code and assign to neighborhood.
df1 = pd.DataFrame(df1.groupby(['Postal code', 'Borough'])['Neighborhood'].apply(lambda x: ', '.join(x))).reset_index()

In [391]:
#change "/" seperator to ","
df1=df1.apply(lambda x: x.str.replace('/',','))

In [392]:
df1

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park"
7,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge"
8,M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff , Cliffside West"


### 2) Get Longitude and Latitude

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [393]:
#read in csv file
data = pd.read_csv("/Users/thanuavirneni/Downloads/Geospatial_Coordinates.csv") 
data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [394]:
#Rename Postal Code to match df2 and data data frames.
df1.rename(columns={'Postal code': 'Postal Code'}, inplace=True)

In [395]:
#Merge dataframes to include latitude and longitude.
df2 = pd.merge(df1, data[["Postal Code","Latitude", "Longitude"]], on="Postal Code", how="left")
df2

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff , Cliffside West",43.692657,-79.264848


### 3) Explore and Cluster the Neighborhoods in Toronto

Explore and cluster the neighborhoods in Toronto.
Just make sure:

1) to add enough Markdown cells to explain what you decided to do and to report any observations you make.

2) to generate maps to visualize your neighborhoods and how they cluster together.

In [396]:
#Explore how many unique boroughs and neighborhoods are present
print('The dataframe has {} boroughs and {} neighborhood.'.format(
        len(df2['Borough'].unique()),
        df2.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhood.


In [397]:
#Use geopy library to get the latitude and longitude values of Toronto, Canada
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada are 43.6534817, -79.3839347.


In [398]:
# creating map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
    
map_toronto

##### Explore NorthYork.

In [399]:
CLIENT_ID = 'MEAM3W22KEUVVPC5BV5354E4K2N3XCA352NMJIRRG1PO1NUB' # your Foursquare ID
CLIENT_SECRET = '4I3EYMQWM3M1PAWHSQEBSMSCSWKQN223QSPTXIWROZLCKUVN' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MEAM3W22KEUVVPC5BV5354E4K2N3XCA352NMJIRRG1PO1NUB
CLIENT_SECRET:4I3EYMQWM3M1PAWHSQEBSMSCSWKQN223QSPTXIWROZLCKUVN


In [400]:
northyork_data = df2[df2['Borough'] == 'North York'].reset_index(drop=True)
northyork_data.head(5)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Fairview , Henry Farm , Oriole",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"York Mills , Silver Hills",43.757490,-79.374714
4,M2M,North York,"Willowdale , Newtonbrook",43.789053,-79.408493


#### Let's create a map of North York and its neighborhoods

In [401]:
#find geographical coordinates
address_ny = 'North York,Toronto'
latitude_ny = 43.761539
longitude_ny = -79.411079
print('The geograpical coordinate of North York are {}, {}.'.format(latitude_ny, longitude_ny))


The geograpical coordinate of North York are 43.761539, -79.411079.


In [402]:
map_ny = folium.Map(location=[latitude_ny, longitude_ny], zoom_start=12)

# add markers to map
for lat, lng, label in zip(northyork_data['Latitude'], northyork_data['Longitude'], northyork_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_ny)  
    
map_ny

#### Top venues near Hillcrest Vilage from North York

In [403]:
neighborhood_lat = northyork_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_long = northyork_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = northyork_data.loc[0, 'Neighborhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_lat, 
                                                               neighborhood_long))

Latitude and longitude values of "Hillcrest Village" are 43.8037622, -79.3634517.


In [404]:
LIMIT = 100
radius = 400
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_ny, longitude_ny, VERSION, radius, LIMIT)

In [405]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea5b5ad1e152c001c567be5'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Willowdale',
  'headerFullLocation': 'Willowdale, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 40,
  'suggestedBounds': {'ne': {'lat': 43.765139003600005,
    'lng': -79.40610370342658},
   'sw': {'lat': 43.75793899639999, 'lng': -79.41605429657342}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '55665a0b498ec5589987b1f7',
       'name': 'Kinka Izakaya',
       'location': {'address': '4775 Yonge Street, Unit #114',
        'crossStreet': 'Sheppard Ave.',
        'lat': 43.76016102214242,
        'lng': -79.40982686116466,
        'labeled

In [406]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [407]:
import json
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,name,categories,lat,lng
0,Kinka Izakaya,Japanese Restaurant,43.760161,-79.409827
1,EatBKK,Thai Restaurant,43.759320,-79.410454
2,Longo’s Sheppard Centre,Grocery Store,43.762221,-79.410762
3,Sushi Moto Sake & Wine Bar,Sushi Restaurant,43.763902,-79.411559
4,Five Guys,Burger Joint,43.762400,-79.410763
5,Poke Eats,Poke Place,43.760060,-79.410862
6,Pizzaiolo,Pizza Place,43.764289,-79.411780
7,Bake Code 烘焙密碼,Bakery,43.763969,-79.411641
8,Hero Certified Burgers,Burger Joint,43.758885,-79.410249
9,Whole Foods Market,Grocery Store,43.760662,-79.411044


In [408]:
print('{} venues'.format(nearby_venues.shape[0]))

40 venues


In [409]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Analyze Each Neighborhood in North York

In [410]:
northyork_venues = getNearbyVenues(names=northyork_data['Neighborhood'],
                                   latitudes=northyork_data['Latitude'],
                                   longitudes=northyork_data['Longitude']
                                  )

Hillcrest Village
Fairview , Henry Farm , Oriole
Bayview Village
York Mills , Silver Hills
Willowdale , Newtonbrook
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills
Bathurst Manor , Wilson Heights , Downsview North
Northwood Park , York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Bedford Park , Lawrence Manor East
Lawrence Manor , Lawrence Heights
Glencairn
North Park , Maple Leaf Park , Upwood Park
Humber Summit
Humberlea , Emery


In [411]:
northyork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor , Wilson Heights , Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park , Lawrence Manor East",23,23,23,23,23,23
Don Mills,26,26,26,26,26,26
Downsview,17,17,17,17,17,17
"Fairview , Henry Farm , Oriole",65,65,65,65,65,65
Glencairn,4,4,4,4,4,4
Hillcrest Village,5,5,5,5,5,5
Humber Summit,3,3,3,3,3,3


In [412]:
northyork_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,New York Fries,43.803664,-79.363905,Fast Food Restaurant
2,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
3,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
4,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run
5,"Fairview , Henry Farm , Oriole",43.778517,-79.346556,The LEGO Store,43.778207,-79.343483,Toy / Game Store
6,"Fairview , Henry Farm , Oriole",43.778517,-79.346556,DAVIDsTEA,43.777593,-79.345089,Tea Room
7,"Fairview , Henry Farm , Oriole",43.778517,-79.346556,Purdys Chocolatier,43.778160,-79.344154,Chocolate Shop
8,"Fairview , Henry Farm , Oriole",43.778517,-79.346556,Michel's Baguette,43.777082,-79.344557,Bakery
9,"Fairview , Henry Farm , Oriole",43.778517,-79.346556,Hero Certified Burgers,43.777295,-79.344584,Burger Joint


In [413]:
# one hot encoding
ny_onehot = pd.get_dummies(northyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot['Neighborhood'] = northyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]

ny_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Service,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Women's Store
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [414]:
ny_onehot.shape

(239, 100)

In [415]:
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()
ny_grouped.head(7)

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Service,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor , Wilson Heights , Downsview North",0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.050000,0.000000,0.050000,0.000000,0.000000,0.05,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.05,0.0,0.05,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.05,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.05,0.0,0.000000,0.0,0.000000,0.0,0.050000,0.050000,0.0,0.0,0.0,0.000000,0.0,0.050000,0.000000,0.050000,0.000000,0.050000,0.000000,0.0,0.050000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,0.0,0.000000
1,Bayview Village,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.250000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000
2,"Bedford Park , Lawrence Manor East",0.0,0.000000,0.043478,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.043478,0.043478,0.000000,0.000000,0.000000,0.000000,0.086957,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.00,0.0,0.043478,0.043478,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.043478,0.0,0.0,0.086957,0.000000,0.043478,0.000000,0.043478,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.0,0.043478,0.043478,0.0,0.0,0.0,0.043478,0.0,0.086957,0.000000,0.086957,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.043478,0.000000,0.043478,0.000000,0.000000,0.000000,0.00,0.0,0.043478
3,Don Mills,0.0,0.000000,0.000000,0.0,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.038462,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.038462,0.038462,0.038462,0.000000,0.038462,0.076923,0.000000,0.038462,0.00

##### top 10 most venues in neighborhood

In [416]:
num_top_venues = 10

for village in toronto_grouped['Neighborhood']:
    print("----"+village+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == village].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                     Lounge   0.2
1  Latin American Restaurant   0.2
2             Breakfast Spot   0.2
3             Clothing Store   0.2
4               Skating Rink   0.2
5          Mobile Phone Shop   0.0
6                     Market   0.0
7          Martial Arts Dojo   0.0
8             Massage Studio   0.0
9             Medical Center   0.0


----Alderwood / Long Branch----
                venue  freq
0         Pizza Place   0.2
1         Coffee Shop   0.1
2                 Gym   0.1
3            Pharmacy   0.1
4      Sandwich Place   0.1
5        Skating Rink   0.1
6        Dance Studio   0.1
7  Athletics & Sports   0.1
8                 Pub   0.1
9         Men's Store   0.0


----Bathurst Manor / Wilson Heights / Downsview North----
                       venue  freq
0                Coffee Shop  0.10
1                       Bank  0.10
2         Frozen Yogurt Shop  0.05
3              Shopping Mall  0.05
4             Sandwi

#### Put data in pandas dataframe

In [417]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [418]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ny_grouped['Neighborhood']

for ind in np.arange(ny_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor , Wilson Heights , Downsview North",Latin American Restaurant,Skating Rink,Clothing Store,Breakfast Spot,Lounge,Department Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
1,Bayview Village,Pizza Place,Dance Studio,Sandwich Place,Coffee Shop,Skating Rink,Pharmacy,Gym,Athletics & Sports,Pub,Deli / Bodega
2,"Bedford Park , Lawrence Manor East",Coffee Shop,Bank,Gas Station,Middle Eastern Restaurant,Diner,Sandwich Place,Bridal Shop,Restaurant,Ice Cream Shop,Supermarket
3,Don Mills,Japanese Restaurant,Bank,Chinese Restaurant,Café,Women's Store,Department Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
4,Downsview,Italian Restaurant,Sandwich Place,Coffee Shop,Restaurant,Thai Restaurant,Pizza Place,Spa,Liquor Store,Indian Restaurant,Butcher


## Clustering Neighborhoods

In [419]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

northyork_data = northyork_data.drop(14)
# set number of clusters
kclusters = 5
ny_grouped_clustering = ny_grouped.drop('Neighborhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 3, 4], dtype=int32)

In [420]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ny_merged = northyork_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ny_merged = ny_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

ny_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,1.0,Café,Breakfast Spot,Coffee Shop,Burrito Place,Italian Restaurant,Intersection,Stadium,Restaurant,Bar,Bakery
1,M2J,North York,"Fairview , Henry Farm , Oriole",43.778517,-79.346556,1.0,Seafood Restaurant,Coffee Shop,Cocktail Bar,Farmers Market,Park,Bakery,Basketball Stadium,Beer Bar,Museum,Jazz Club
2,M2K,North York,Bayview Village,43.786947,-79.385975,1.0,Pizza Place,Dance Studio,Sandwich Place,Coffee Shop,Skating Rink,Pharmacy,Gym,Athletics & Sports,Pub,Deli / Bodega
3,M2L,North York,"York Mills , Silver Hills",43.757490,-79.374714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M2M,North York,"Willowdale , Newtonbrook",43.789053,-79.408493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Visualize Clusters

In [421]:
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location = [latitude_ny, longitude_ny], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighborhood'], ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #color=rainbow[cluster-1],
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

##### Cluster 0

In [422]:
ny_merged.loc[ny_merged['Cluster Labels'] == 0, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,North York,0.0,Pharmacy,Pizza Place,Coffee Shop,Chinese Restaurant,Bank,Fast Food Restaurant,Thai Restaurant,Italian Restaurant,Convenience Store,Fried Chicken Joint
21,North York,0.0,Coffee Shop,Hotel,American Restaurant,Fried Chicken Joint,Middle Eastern Restaurant,Mediterranean Restaurant,Gym,Burrito Place,Sandwich Place,Intersection


##### Cluster 1

In [423]:
ny_merged.loc[ny_merged['Cluster Labels'] == 1, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1.0,Café,Breakfast Spot,Coffee Shop,Burrito Place,Italian Restaurant,Intersection,Stadium,Restaurant,Bar,Bakery
1,North York,1.0,Seafood Restaurant,Coffee Shop,Cocktail Bar,Farmers Market,Park,Bakery,Basketball Stadium,Beer Bar,Museum,Jazz Club
2,North York,1.0,Pizza Place,Dance Studio,Sandwich Place,Coffee Shop,Skating Rink,Pharmacy,Gym,Athletics & Sports,Pub,Deli / Bodega
5,North York,1.0,Salon / Barbershop,Thai Restaurant,Juice Bar,Ramen Restaurant,Pub,Hobby Shop,Diner,Smoke Shop,Café,Sushi Restaurant
7,North York,1.0,Salon / Barbershop,Thai Restaurant,Juice Bar,Ramen Restaurant,Pub,Hobby Shop,Diner,Smoke Shop,Café,Sushi Restaurant
9,North York,1.0,Japanese Restaurant,Bank,Chinese Restaurant,Café,Women's Store,Department Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
10,North York,1.0,Japanese Restaurant,Bank,Chinese Restaurant,Café,Women's Store,Department Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
11,North York,1.0,Latin American Restaurant,Skating Rink,Clothing Store,Breakfast Spot,Lounge,Department Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
12,North York,1.0,Thai Restaurant,Bakery,Athletics & Sports,Gas Station,Caribbean Restaurant,Hakka Restaurant,Fried Chicken Joint,Bank,Dim Sum Restaurant,Dessert Shop
13,North York,1.0,Italian Restaurant,Sandwich Place,Coffee Shop,Restaurant,Thai Restaurant,Pizza Place,Spa,Liquor Store,Indian Restaurant,Butcher


#### Cluster 2

In [424]:
ny_merged.loc[ny_merged['Cluster Labels'] == 2, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,North York,2.0,Coffee Shop,Café,Sushi Restaurant,Bar,Spa,Sandwich Place,Bubble Tea Shop,Ramen Restaurant,Park,Yoga Studio


#### Cluster 3

In [425]:
ny_merged.loc[ny_merged['Cluster Labels'] == 3, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,North York,3.0,Light Rail Station,Yoga Studio,Auto Workshop,Smoke Shop,Spa,Brewery,Burrito Place,Farmers Market,Fast Food Restaurant,Restaurant


#### Cluster 4

In [426]:
ny_merged.loc[ny_merged['Cluster Labels'] == 4, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,North York,4.0,Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Harbor / Marina,Plane,Rental Car Location,Sculpture Garden,Boutique,Bar
